## Raspagem com python e beautiful soup

In [221]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [222]:
meses = list(range(1,7,1))
dias = list(range(1,32,1))
lista_URL = []

for mes in meses:
  for dia in dias:
    if dia < 10:
      URL = f"https://www.gov.br/planalto/pt-br/acompanhe-o-planalto/agenda-do-presidente-da-republica-lula/agenda-do-presidente-da-republica/2023-0{mes}-0{dia}"
    else:
      URL = f"https://www.gov.br/planalto/pt-br/acompanhe-o-planalto/agenda-do-presidente-da-republica-lula/agenda-do-presidente-da-republica/2023-0{mes}-{dia}"
    lista_URL.append(URL)

In [229]:
lista_atividades = []

for i in lista_URL:
  response = requests.get(i)
  if response.status_code == 200:
    html = response.text
    soup = BeautifulSoup(html, "html.parser")
    results = soup.find_all("ul", class_="list-compromissos")
    datas_compromissos = soup.find_all("li", class_="day is-selected has-appointment")

  # coletar o dia exato do compromisso

    lista_datas = []

    for i in datas_compromissos:
      link = i.find('a', href=True)
      link_txt = link['href'][-10:]

    for compromisso in results:

      # coletar os elementos que compõem cada compromisso na agenda

      elemento_compromisso = compromisso.find_all("li", class_="item-compromisso-wrapper")

      for compromisso in elemento_compromisso:
        atividade = {}

        horario = compromisso.find("time", class_="compromisso-inicio")
        titulo = compromisso.find("h2", class_="compromisso-titulo")
        local = compromisso.find("div", class_="compromisso-local")
        atividade['horario'] = horario.string
        atividade['data'] = link_txt
        atividade['titulo'] = titulo.string
        if local != None:
          atividade['local'] = local.string
        else:
          atividade['local'] = local

        lista_atividades.append(atividade)

In [230]:
df = pd.DataFrame(lista_atividades)

In [231]:
df

,horario,data,titulo,local
0,15h00,2023-01-01,Solenidade de Posse Presidencial no Congresso ...,Congresso Nacional
1,17h00,2023-01-01,Presidente da República recebe a faixa preside...,Palácio do Planalto
2,17h10,2023-01-01,Discurso no Parlatório,Palácio do Planalto
3,17h50,2023-01-01,Cumprimentos dos chefes de Estado e de Governo,Palácio do Planalto
4,19h10,2023-01-01,Assinatura de atos,Palácio do Planalto
...,...,...,...,...
833,13h30,2023-06-30,Almoço oferecido pelo Governador do Rio Grande...,Palácio Piratini
834,15h30,2023-06-30,Visita às novas instalações do Hospital de Clí...,Hospital de Clínicas de Porto Alegre
835,16h00,2023-06-30,Solenidade de inauguração formal dos novos blo...,Hospital de Clínicas de Porto Alegre
836,17h45,2023-06-30,Partida para Brasília,Aeroporto Salgado Filho
